In [1]:
import numpy as np
from datetime import datetime
import requests
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
url = "http://vps-40d69db1.vps.ovh.ca:22001/api/ExecucaoSimulacaos/melhorValidacaoPeriodoFlat?idPeriodo=9"

In [3]:
r = requests.get(url)
j = r.json()
df = pd.DataFrame.from_dict(j)

In [4]:
df.columns

Index(['simulacaoId', 'ticker', 'simulacaoResultado', 'simulacaoLucro',
       'simulacaoPrejuizo', 'target', 'stop', 'tipo', 'experimentoSimulacaoId',
       'validacaoLucro', 'validacaoPrejuizo', 'saldoValidacao'],
      dtype='object')

Qual o período ?
Quantos experimentos ?
Total de casos ?

In [5]:
dfResumo = df.groupby('experimentoSimulacaoId').count()
dfResumo = dfResumo.filter(items=['ticker', 'validacaoLucro','saldoValidacao'])
dfResumo['percentual'] = (dfResumo['validacaoLucro'] / dfResumo['ticker']) * 100
dfResumo.head()

,ticker,validacaoLucro,saldoValidacao,percentual
experimentoSimulacaoId,,,,
54,14458,14422,14422,99.751003
56,11339,11329,11329,99.911809
57,27537,27528,27528,99.967317
58,13303,13287,13287,99.879726


Tickers 

In [6]:
print("Quantidade total de tickers: %2d "%(len(df['ticker'].unique())))

Quantidade total de tickers: 60 


In [7]:
dfTicker = df.groupby(['ticker','saldoValidacao']).size().reset_index(name ='qtde')

In [8]:
dfTicker.head()

,ticker,saldoValidacao,qtde
0,AERI3,0.0,227
1,AERI3,1.0,22
2,AERI3,2.0,50
3,ALPA4,0.0,695
4,ALSO3,0.0,21


In [9]:
print("Quantidade de tickers já tratados: %2d "%(len(dfTicker['ticker'].unique())))

Quantidade de tickers já tratados: 60 


In [10]:
dfTicker2 = dfTicker.pivot_table('qtde','ticker','saldoValidacao')
dfTicker2.fillna(0, inplace=True)
dfTicker2['positivo'] = dfTicker2[1.0] + dfTicker2[2.0]
dfTicker2['negativo'] = dfTicker2[-1.0] + dfTicker2[-2.0] + dfTicker2[-3.0]
dfTicker2.drop(columns=[-3.0, -2.0, -1.0, 1.0, 2.0], inplace=True)
dfTicker2.head(11)

saldoValidacao,0.0,positivo,negativo
ticker,,,
AERI3,227.0,72.0,0.0
ALPA4,695.0,0.0,0.0
ALSO3,21.0,120.0,0.0
AMAR3,658.0,155.0,261.0
ANIM3,364.0,68.0,1182.0
AZUL4,240.0,72.0,0.0
B3SA3,3300.0,405.0,0.0
BKBR3,705.0,1107.0,0.0
BPAC11,5371.0,0.0,2217.0


In [11]:

dfBom = dfTicker2[(dfTicker2['negativo']==0) & (dfTicker2['positivo']>0)]

In [12]:
len(dfBom)

20

In [13]:
dfBom.head(25)

saldoValidacao,0.0,positivo,negativo
ticker,,,
AERI3,227.0,72.0,0.0
ALSO3,21.0,120.0,0.0
AZUL4,240.0,72.0,0.0
B3SA3,3300.0,405.0,0.0
BKBR3,705.0,1107.0,0.0
BPAN4,2283.0,157.0,0.0
BRAP4,0.0,82.0,0.0
BRML3,2280.0,72.0,0.0
CVCB3,0.0,252.0,0.0


# Relacionar isso com os dados de regra-projecao.

### <<dataframe>>resultadoTickerComRegraProjecao(in regraProjecao, in dataInicio)

In [17]:
url2 = "http://vps-40d69db1.vps.ovh.ca:22001/api/RegraProjecaos/tickerPerformance?dataNumInicio=20211001&dataNumFinal=20211030&grupoAcao=Neg-100&codigoRegra=1"

In [18]:
r = requests.get(url2)
j = r.json()
df2 = pd.DataFrame.from_dict(j)
df2.set_index('ticker', inplace=True)

In [20]:
df2.head(30)

,ticks,positivos,negativos,zero
ticker,,,,
ABEV3,300,270,0,30
AERI3,300,0,0,0
ALPA4,300,0,0,0
ALSO3,300,0,0,0
AMAR3,300,0,0,0
ANIM3,300,0,0,0
AZUL4,300,14,286,0
B3SA3,300,0,78,222
BBAS3,300,48,0,252


In [21]:
dfjoin = df2.join(dfTicker2)

In [24]:
dfVendas = dfjoin.sort_values(by = 'positivo', ascending = False)

In [25]:
dfVendas.head(30)

,ticks,positivos,negativos,zero,0.0,positivo,negativo
ticker,,,,,,,
LWSA3,300,16,284,0,847.0,2082.0,286.0
WIZS3,300,0,300,0,128.0,1438.0,0.0
ECOR3,300,49,174,77,12.0,1401.0,0.0
BKBR3,300,0,0,0,705.0,1107.0,0.0
YDUQ3,300,112,163,25,118.0,503.0,0.0
ELET3,300,0,229,71,126.0,435.0,0.0
MULT3,300,146,104,50,603.0,416.0,0.0
B3SA3,300,0,78,222,3300.0,405.0,0.0
RAPT4,300,0,0,0,983.0,390.0,0.0


In [26]:
dfVendas['negativos'].corr(dfVendas['positivo'])

0.1612422883535427